# pytorch简单实现线性模型

In [19]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [15]:
features.shape,labels.shape

(torch.Size([1000, 2]), torch.Size([1000]))

In [17]:
import torch.utils.data as Data   # data包用来读取数据
batch_size = 10
dataset = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

## 继承nn.Module，来定义自己的网络层
1. 包含一些层
2. 包含forward

In [40]:
from torch import nn
class LinearNet(nn.Module):   # 继承
    def __init__(self,n_features):
        super().__init__()   # python3可以直接这样写了
        # super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_features,1)

    def forward(self,x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net)
    


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


打印net的参数

In [45]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0094,  0.0056]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


权重的初始化，借助init模块

In [43]:
from torch.nn import init

init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

损失函数和优化算法

In [47]:
loss = nn.MSELoss()

import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


训练模型

In [48]:
num_epochs = 3
for epoch in range(1,num_epochs + 1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output,y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()   # 使用step来迭代模型参数，可以指明批量大小（miniSGD用来对样本梯度求平均）
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000204
epoch 2, loss: 0.000135
epoch 3, loss: 0.000113


In [50]:
dense = net.linear
print(true_w, dense.weight)
print(true_b, dense.bias)


[2, -3.4] Parameter containing:
tensor([[ 2.0003, -3.4007]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1989], requires_grad=True)
